# 지도학습- 거리기반(KNN)

In [1]:
from hossam import *

from pandas import DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve

# 성능 평가 지표 모듈
from sklearn.metrics import(
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error
)

# 확률적 경사 하강법
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

c:\Users\wodyd\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📦 아이티윌 이광호 강사가 제작한 라이브러리를 사용중입니다.
📚 자세한 사용 방법은 https://py.hossam.kr 을 참고하세요.
📧 Email: leekh4232@gmail.com
🎬 Youtube: https://www.youtube.com/@hossam-codingclub
📝 Blog: https://blog.hossam.kr/
🔖 Version: 0.4.18

✅ 시각화를 위한 한글 글꼴(NotoSansKR-Regular)이 자동 적용되었습니다.


In [3]:
origin = load_data('restaurant_sales_preprocessed')
origin.set_index('date', inplace = True)
origin['holiday'] = origin['holiday'].astype('category')
origin['weekend'] = origin['weekend'].astype('category')
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print(origin.info())
origin.head()

어느 식당의 1년간 일별 매출을 기록한 데이터의 전처리 완료 버전(명목형이 이진변수만 있으므로 더미변수는 처리하지 않음)
데이터셋 크기: (353, 9)
열 개수: 9
행 개수: 353
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 353 entries, 2024-01-01 to 2024-12-30
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   sales           353 non-null    float64 
 1   visitors        353 non-null    int64   
 2   avg_price       353 non-null    int64   
 3   marketing_cost  353 non-null    float64 
 4   delivery_ratio  353 non-null    float64 
 5   rain_mm         353 non-null    float64 
 6   temperature     353 non-null    float64 
 7   holiday         353 non-null    category
 8   weekend         353 non-null    category
dtypes: category(2), float64(5), int64(2)
memory usage: 23.0 KB
None


,sales,visitors,avg_price,marketing_cost,delivery_ratio,rain_mm,temperature,holiday,weekend
date,,,,,,,,,
2024-01-01,16.169336,122,14490,9.200391,0.48,1.945910,5.2,0,0
2024-01-02,16.005083,106,11880,9.148571,0.48,2.079442,11.1,0,0
2024-01-03,16.353561,120,18010,8.665786,0.43,2.595255,12.9,1,0
2024-01-04,16.082478,115,11160,8.748464,0.41,2.028148,12.9,0,0
2024-01-05,16.113222,124,15480,8.594339,0.51,2.197225,11.0,0,0


## 1. 훈련, 검증 데이터 분리

In [5]:
df = origin

yname = 'sales'
x = df.drop(columns = [yname])
y = df[yname]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.25, random_state = 52
)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((264, 8), (89, 8), (264,), (89,))

## 2. 모형 적합

In [6]:
%%time

# 파이프라인 정의
pipe = Pipeline(
    [('scaler', StandardScaler()), ('model', KNeighborsRegressor())]
)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    # 핵심 파라미터
    'model__n_neighbors': [3, 5, 7, 9, 15, 25],
    'model__weights': ['uniform', 'distance'],
    # 거리 정의
    'model__metric': ['euclidean','manhattan','minkowski'],
    'model__p': [
        1,
        2
    ], # p=1: Manhattan, p=2: Eculidean -> metric ='minkowski'일 때만 p 의미 있음    
    # 탐색 알고리즘 / 성능
    'model__algorithm': ['auto', 'ball_tree','kd_tree'],
    'model__leaf_size': [20, 30, 40]
}

# 그리드 서치 객체 생성
gs = GridSearchCV(estimator = pipe, param_grid = param_grid, cv = 5, scoring = 'r2', n_jobs = -1)

gs.fit(x_train, y_train)

# 최적 모델
best_model = gs.best_estimator_

# 결과 확인
print('Best params:', gs.best_params_)
print('CV best score:', gs.best_score_)

Best params: {'model__algorithm': 'auto', 'model__leaf_size': 20, 'model__metric': 'euclidean', 'model__n_neighbors': 15, 'model__p': 1, 'model__weights': 'distance'}
CV best score: 0.6444866391028172
CPU times: total: 2.5 s
Wall time: 17.4 s


## 3. 성능평가 지표

In [9]:
y_pred = best_model.predict(x_test)

result_df = DataFrame(
    {
        '결정계수(R2)': r2_score(y_test, y_pred),
        '평균절대오차(MAE)': mean_absolute_error(y_test, y_pred),
        '평균제곱오차(MSE)': mean_squared_error(y_test, y_pred),
        '평균오차(RMSE)': np.sqrt(mean_squared_error(y_test, y_pred)),
        '평균 절대 백분오차 비율(MAPE)': mean_absolute_percentage_error(y_test, y_pred),
        '평균 비율 오차(MPE)': np.mean((y_test - y_pred) / y_test * 100)
    },
    index = ['성능지표']
)

display(result_df)

,결정계수(R2),평균절대오차(MAE),평균제곱오차(MSE),평균오차(RMSE),평균 절대 백분오차 비율(MAPE),평균 비율 오차(MPE)
성능지표,0.634869,0.178029,0.052234,0.228547,0.010843,0.301152


## 4. 과적합 확인 함수

In [ ]:
hs_stats

SyntaxError: invalid syntax (2977448628.py, line 1)